# SparseEdges : computing sparseness of natural images with retina-like RFs

Let's compute the "edges" produced with symmetrical filters.

## Initialization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cluster = True

In [3]:
from __future__ import division, print_function
import matplotlib
pylab_defaults = { 
    'font.size': 10,
    'xtick.labelsize':'medium',
    'ytick.labelsize':'medium',
    'text.usetex': False,
    'font.family' : 'sans-serif',
    'font.sans-serif' : ['Helvetica'],
    }
matplotlib.rcParams.update(pylab_defaults)

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
#%config InlineBackend.figure_format = 'svg'
import os
import numpy as np
np.set_printoptions(precision=2, suppress=True)
fig_width_pt = 397.48  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
fig_width = fig_width_pt*inches_per_pt  # width in inches
#fig_width = 21
figsize=(fig_width, .618*fig_width)

In [4]:
%cd ../test
figpath, ext = os.path.join(os.getenv('HOME'), 'pool/science/RetinaClouds/2016-05-20_nips'), '.pdf'
exp='retina_sparseness'

/Users/laurentperrinet/pool/science/BICV/SparseEdges/test


defining framework


In [5]:
from SparseEdges import SparseEdges
mp = SparseEdges('https://raw.githubusercontent.com/meduz/SparseEdges/master/default_param.py')
print ('Range of spatial frequencies: ', mp.sf_0)

AttributeError: 'ParameterSet' object has no attribute 'mask_exponent'

Standard edges are oriented, but one may modify that:

In [ ]:
sf_0 = .02 # TODO .1 cycle / pixel (Geisler)
params= {'sf_0':sf_0, 'B_sf': mp.pe.B_sf, 'theta':0., 'B_theta': mp.pe.B_theta}
FT_lg = mp.loggabor(mp.N_X/2, mp.N_Y/2, **params)
#(fourier_domain(mp.normalize(np.absolute(FT_lg), center=False))+ image_domain(mp.normalize(mp.invert(FT_lg), center=False)))
fig, a1, a2 = mp.show_FT(FT_lg, axis=True)

In [ ]:
sf_0 = .02 # TODO .1 cycle / pixel (Geisler)
params= {'sf_0':sf_0, 'B_sf': mp.pe.B_sf, 'theta':0., 'B_theta': np.inf}
FT_lg = mp.loggabor(mp.N_X/2, mp.N_Y/2, **params)
fig, a1, a2 = mp.show_FT(FT_lg, axis=True)
fig.savefig(os.path.join(figpath, exp + '_dog' + ext))

When defining the framework, one thus needs only one angle:

In [ ]:
print ('Range of angles: ', mp.theta*180./np.pi)
mp.pe.n_theta = 1
mp.pe.B_theta = np.inf
mp.init()
print ('Range of angles: ', mp.theta*180./np.pi)

In [ ]:
print('Final sparseness in the pyramid = {}'.format(mp.pe.N/(4/3*mp.N_X*mp.N_Y)))

## one example image

In [ ]:
image = mp.imread('https://raw.githubusercontent.com/meduz/SparseEdges/master/database/lena256.png')
mp.pe.mask_exponent = 4.
mp.init()
image *= mp.mask
image = mp.normalize(image, center=True)
fig, axs[i_ax] = mp.imshow(image, mask=True)


In [ ]:
mp.pe.N = 2**14
name = exp.replace('_sparseness', '_lena')
matname = os.path.join(mp.pe.matpath, name + '.npy')
try:
    edges = np.load(matname)
except:
    edges, C_res = mp.run_mp(image, verbose=False)
    np.save(matname, edges)    

image_rec = mp.reconstruct(edges, mask=True)        

In [ ]:
mp.pe.line_width = 0
fig, a = mp.show_edges(edges, image=mp.dewhitening(image_rec), show_phase=False, mask=True)
fig.savefig(os.path.join(figpath, name + ext))

In [ ]:
for i in range(15): print(np.logspace(1, 13, i, base=2))

In [ ]:
list_of_number_of_edge = np.logspace(1, 13, 7, base=2) *2
fig, axs = plt.subplots(1, len(list_of_number_of_edge), figsize=(3*fig_width, 3*fig_width/len(list_of_number_of_edge)))
vmax = 1.
image_rec = mp.reconstruct(edges, mask=True)        
vmax = mp.dewhitening(image_rec).max()
for i_ax, number_of_edge in enumerate(list_of_number_of_edge):
    edges_ = edges[:, :number_of_edge][..., np.newaxis]
    image_rec = mp.dewhitening(mp.reconstruct(edges_, mask=True))
    fig, axs[i_ax] = mp.imshow(image_rec/vmax, fig=fig, ax=axs[i_ax], norm=False, mask=True)
    axs[i_ax].text(5, 29, 'N=%d' % number_of_edge, color='red', fontsize=24)
plt.tight_layout()
fig.subplots_adjust(hspace = .0, wspace = .0, left=0.0, bottom=0., right=1., top=1.)

if not(figpath==''): 
    fig.savefig(os.path.join(figpath, exp + '_lena_movie' + ext))

## Running simulations on a set of natural images

In [ ]:
%%writefile ../test/experiment_retina_sparseness.py
# -*- coding: utf8 -*-
from __future__ import division, print_function
"""

$ python experiment_retina_sparseness.py

rm -fr **/retina_sparseness* **/**/retina_sparseness*

"""
import numpy as np
from SparseEdges import SparseEdges
mps = []
for name_database in ['serre07_distractors']:#, 'serre07_distractors_urban', 'laboratory']:
    mp = SparseEdges('https://raw.githubusercontent.com/meduz/SparseEdges/master/default_param.py')
    mp.pe.datapath = '../../SLIP/database/'
    mp.pe.N_image = 100
    mp.pe.N = 2**14
    mp.pe.n_theta = 1
    mp.pe.B_theta = np.inf
    mp.pe.line_width = 0
    mp.init()
    # normal experiment
    imageslist, edgeslist, RMSE = mp.process(exp='retina_sparseness', name_database=name_database)
    mps.append(mp)
    # control experiment
    mp.pe.MP_alpha = np.inf
    mp.init()
    imageslist, edgeslist, RMSE = mp.process(exp='retina_sparseness_linear', name_database=name_database)
    mps.append(mp)


In [ ]:
%run experiment_retina_sparseness.py

In [ ]:
SERVER = 'perrinet.l@frioul.int.univ-amu.fr'
PATH = '/hpc/invibe/perrinet.l/science/SparseEdges/test/'
def run_on_cluster(cmd, PATH=PATH, SERVER=SERVER):
    import subprocess
    fullcmd = 'ssh {SERVER} "cd {PATH} ; {cmd} "'.format(SERVER=SERVER, PATH=PATH, cmd=cmd)
    print ('⚡︎ Running ⚡︎ ', fullcmd)
    stdout = subprocess.check_output([fullcmd], shell=True)
    return stdout.decode().splitlines()
if cluster:
    for cmd in [
        #"cd ..; make update_dev",
        #"rm -fr **/retina_sparseness* **/**/retina_sparseness*",
        "find . -name *lock* -exec rm -fr {} \\;",
        "find . -name *retina_sparseness_linear* -exec rm -fr {} \\;",
        "rm frioul* ",
        "frioul_batch  -M 136 'ipython experiment_retina_sparseness.py' ", 
        "frioul_list_jobs -v |grep job_array_id |uniq -c",
                ]:
        print(run_on_cluster(cmd))

In [ ]:
if cluster:
    for cmd in [
        "frioul_list_jobs -v |grep job_array_id |uniq -c",
    ]:
        print(run_on_cluster(cmd))

In [ ]:
import time
time.sleep(4800)

In [ ]:
def fetch_from_cluster(source="{figures,mat}", dest=".", PATH=PATH, SERVER=SERVER, opts="-av --exclude .AppleDouble --exclude .git"):
    import subprocess
    fullcmd = 'rsync {opts} {SERVER}:{PATH}{source} {dest}  '.format(opts=opts, SERVER=SERVER, PATH=PATH, source=source, dest=dest)
    print ('⚡︎ Running ⚡︎ \n', fullcmd)
    stdout = subprocess.check_output([fullcmd], shell=True)
    return stdout.decode().splitlines()
#cluster = True
if cluster: fetch_from_cluster()

## Analysing results


First, we retrieve edges from a prior edge extraction

In [ ]:
experiment = 'retina_sparseness'
name_database='serre07_distractors'
imageslist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)


In [ ]:
fig, [A, B] = plt.subplots(1, 2, figsize=(fig_width, fig_width/1.618), subplot_kw={'axisbg':'w'})
help(A.set_color_cycle)

In [ ]:
fig, [A, B] = plt.subplots(1, 2, figsize=(fig_width, fig_width/1.618), subplot_kw={'axisbg':'w'})
A.set_color_cycle(np.array([[1., 0., 0.]]))
imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
RMSE /= RMSE[:, 0][:, np.newaxis]
#print RMSE.shape, edgeslist.shape
value = edgeslist[4, ...]
#value /= value[0, :][np.newaxis, :]
value /= RMSE[:, 0][np.newaxis, :]

B.semilogx( value, alpha=.7)

A.semilogx( RMSE.T, alpha=.7)
A.set_xlabel('l0')
B.set_xlabel('l0')
A.axis('tight')
B.axis('tight')
_ = A.set_ylabel('RMSE')


#plt.locator_params(axis = 'x', nbins = 5)
#plt.locator_params(axis = 'y', nbins = 5)

fig.savefig(os.path.join(figpath, exp + '_raw' + ext))

In [ ]:
fig = plt.figure(figsize=(fig_width/1.618, fig_width/1.618))
fig, a, ax = mp.plot(mps=mps, experiments=[experiment, experiment + '_linear'], databases=[name_database, name_database], fig=fig, 
                  color=[0., 0., 1.], scale=False, labels=['MP', 'lin'])#
fig.savefig(os.path.join(figpath, exp + '_raw_inset' + ext))

## trying different fits

### on the modulation function
!pip install lmfit

In [ ]:
imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
value = edgeslist[4, ...].T
#value /= RMSE[:, 0][np.newaxis, :]
value /= RMSE[:, 0][:, np.newaxis]
#RMSE /= RMSE[:, 0][:, np.newaxis]
N_image, N = RMSE.shape #number of images x edges
#value = value.T

In [ ]:
from lmfit.models import ExpressionModel
mod = ExpressionModel('amplitude * exp ( - .5 * log(x+1)**2 / log(rho+1) **2 )')
verbose = False
amplitude, rho = np.zeros(N_image), np.zeros(N_image)
for i_image in range(N_image):
    #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
    mod.def_vals = {'amplitude':.01, 'rho':100}
    params = mod.make_params()
    out  = mod.fit(value[i_image, :], x=np.arange(N), verbose=verbose)
    #print(out.fit_report())
    amplitude[i_image] = out.params.get('amplitude').value
    rho[i_image] =  out.params.get('rho').value

In [ ]:
amplitude, rho

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/1.618), subplot_kw={'axisbg':'w'})

for i_image in range(N_image):
    ax.loglog( value[i_image, :], alpha=.2)
    params = mod.make_params(amplitude=amplitude[i_image], rho=rho[i_image])
    ax.loglog(mod.eval(params, x=np.arange(N)), 'r--', alpha=.2)
    ax.set_xlabel('l0')
    ax.axis('tight')
    _ = ax.set_ylabel('coefficient')            
fig.savefig(os.path.join(figpath, exp + '_fit_all' + ext))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(fig_width, fig_width/1.618), subplot_kw={'axisbg':'w'})

for i_image in range(N_image):
    ax.semilogy(np.log((np.arange(N)+1)/np.log(rho[i_image]+1)), value[i_image, :]/amplitude[i_image], alpha=.2)
    params = mod.make_params(amplitude=amplitude[i_image], rho=rho[i_image])
    ax.semilogy(np.log((np.arange(N)+1)/np.log(rho[i_image]+1)), mod.eval(params, x=np.arange(N))/amplitude[i_image], 'r--', alpha=.2)
    ax.set_xlabel('l0 norm')
    ax.axis('tight')
    _ = ax.set_ylabel('norm. coefficient')            
fig.savefig(os.path.join(figpath, exp + '_fit_norm' + ext))

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(fig_width, fig_width/1.618), subplot_kw={'axisbg':'w'})

axs[0].hist(amplitude)
axs[1].hist(np.abs(rho))
axs[2].scatter(amplitude, np.abs(rho))
for ax in axs: 
    ax.axis('tight')
    _ = ax.set_ylabel('')            
    _ = ax.set_yticks([])            
axs[0].set_ylabel('probability')            
axs[0].set_xlabel('amplitude')
axs[1].set_xlabel('rho')
axs[2].set_xlabel('amplitude')
axs[2].set_ylabel('rho')
fig.tight_layout()
fig.savefig(os.path.join(figpath, exp + '_fit_hist' + ext))


### on the pdf

In [ ]:
help(np.histogram)

In [ ]:
value.max(axis=1).shape

In [ ]:
%pwd


In [ ]:
#imagelist, edgeslist, RMSE = mp.process(exp=experiment + '_linear', name_database=name_database)
#imagelist, edgeslist, RMSE = mp.process(exp=experiment, name_database=name_database)
edgeslist = np.load('mat/edges/retina_sparseness_serre07_distractors_edges.npy')
value = edgeslist[4, ...].T
#value /= RMSE[:, 0][np.newaxis, :]
value /= value.min(axis=1)[:, np.newaxis]
#RMSE /= RMSE[:, 0][:, np.newaxis]
N_image, N = value.shape #number of images x edges
#value = value.T

In [ ]:
N_bins, a_max = 128, value.max()
start, end = N_bins/16, N_bins
print(a_max)
v_hist = np.zeros((N_image, N_bins))
#bins = np.linspace(0, a_max, N_bins+1, endpoint=True)#[:-1]
#print(bins.shape)
for i_image in range(N_image):
    #v_hist[i_image, : ], v_bins = np.histogram(value[i_image, :], bins=bins) 
    v_hist[i_image, : ], v_bins = np.histogram(value[i_image, :], bins=N_bins) 
    v_hist[i_image, : ] /= v_hist[i_image, : ].sum()
print(v_bins.shape)
v_middle = .5*(v_bins[1:]+v_bins[:-1])

In [ ]:
plt.plot(v_bins[1:], v_middle)
print(v_bins[0], v_middle[0])
print(v_bins[-1], v_middle[-1])

MLE estimate of rho:
https://en.wikipedia.org/wiki/Power_law#Maximum_likelihood


In [ ]:
amplitude, rho = np.zeros(N_image), np.zeros(N_image)
for i_image in range(N_image):
    rho[i_image] =  1 +  (end-start) / np.sum(np.log(value[i_image, start:end]))
    amplitude[i_image] = rho[i_image] - 1
print(rho)

In [ ]:
from lmfit.models import ExpressionModel
mod = ExpressionModel('amplitude * x**-rho ')
#mod = ExpressionModel('amplitude * exp( - log(x)**2/rho**2 ) ')
#mod = ExpressionModel('amplitude * exp( - x/rho ) ')
verbose = False
for i_image in range(N_image):
    #pars = mod.guess(RMSE[i_image, :], x=np.arange(N))
    mod.def_vals = {'amplitude': amplitude[i_image], 'rho': rho[i_image]}
    params = mod.make_params()
    out  = mod.fit(v_hist[i_image, start:end], x=v_middle[start:end], verbose=verbose)
    #print(out.fit_report())
    amplitude[i_image] = out.params.get('amplitude').value
    rho[i_image] =  out.params.get('rho').value
print(rho)    

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3*fig_width, 3*fig_width/1.618), subplot_kw={'axisbg':'w'})

for i_image in range(N_image):
    ax.plot(v_middle, v_hist[i_image, :], alpha=.2)
    params = mod.make_params(amplitude=amplitude[i_image], rho=rho[i_image])
    ax.plot(v_middle[start:end], mod.eval(params, x=v_middle[start:end]), 'r.', alpha=.2)
    if True:
      ax.set_yscale('log')
      ax.set_xscale('log')
    ax.set_xlabel('density')
    ax.axis('tight')
    ax.set_xlabel('coefficient')            
fig.savefig(os.path.join(figpath, exp + '_proba' + ext))

In [ ]:
plt.hist(rho)

## some book keeping for the notebook

In [ ]:
%install_ext https://raw.githubusercontent.com/rasbt/python_reference/master/ipython_magic/watermark.py
%load_ext watermark
%watermark

In [ ]:
%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%version_information numpy, scipy, matplotlib, sympy